## Colab Setup

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
"""
민동전용 cd
"""
%cd /content/drive/MyDrive/MLVU/project

[Errno 2] No such file or directory: '/content/drive/MyDrive/MLVU/project'
/content/drive/MyDrive/Colab Notebooks


In [2]:
"""
Change directory to where this file is located
"""
%cd /content/drive/MyDrive/Colab Notebooks

/content/drive/MyDrive/Colab Notebooks


In [ ]:
tf.test.gpu_device_name()

'/device:GPU:0'

## Import Modules:



In [3]:
import tensorflow as tf
import json
from tensorflow.keras.layers import Conv1D,Input,ZeroPadding2D,BatchNormalization,AveragePooling2D, Conv2D,Dropout,Normalization,Softmax
from tqdm import tqdm
import numpy as np
import os
from scipy import special
from sklearn.utils import shuffle
from sklearn import preprocessing
from tensorflow.keras import regularizers
from matplotlib import pyplot as plt

In [ ]:
ftrain_x="nobbr_train_x" 
fvalid_x="nobbr_valid_x"
ftest_x="nobbr_test_x"
ftrain_labels ="nobbr_train_labels"
fvalid_labels ="nobbr_valid_labels"
ftest_labels="nobbr_test_labels"
data_path = './Pose_Dataset/'

channel=256

checkpoint_path = "ck1/cp.ckpt"  #reference model weight이 저장되어 있음.
checkpoint_path2="ck2/cp.ckpt"   #reference model learning rate 다르게 하면서 update 하는 용도
checkpoint_path3="ck3/cp.ckpt"  #dense model weight이 저장되어 있음.
checkpoint_path4="ck4/cp.ckpt"  #dense model learning rate 다르게 하면서 update 하는 용도
checkpoint_dir = os.path.dirname(checkpoint_path)

#At First Loading

## Data Load



In [ ]:
!apt install unzip

Reading package lists... Done
Building dependency tree       
Reading state information... Done
unzip is already the newest version (6.0-25ubuntu1.1).
0 upgraded, 0 newly installed, 0 to remove and 22 not upgraded.


In [ ]:
!ls "/content/drive/My Drive/Colab Notebooks/"

In [ ]:
!unzip -q "/content/drive/My Drive/Colab Notebooks/Pose_Dataset.zip"


replace __MACOSX/._Pose_Dataset? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


In [ ]:
import json


labels=['bad_back_round','bad_back_warp','bad_head','bad_innner_thigh','bad_shallow','bad_toe','good']
paths={'bad_back_round':0,
       'bad_back_warp':0,
       'bad_head':0,
       'bad_innner_thigh':0,
       'bad_shallow':0,
       'bad_toe':0,
       'good':0}

for label in labels:
    paths[label]=data_path+label+"/1115_3djoints_index"


json_files={'bad_back_round':0,
       'bad_back_warp':0,
       'bad_head':0,
       'bad_innner_thigh':0,
       'bad_shallow':0,
       'bad_toe':0,
       'good':0}


for label in labels:
    json_files[label]=[pos_json for pos_json in os.listdir(paths[label]) if pos_json.endswith('.json')]

data={'bad_back_round':[],'bad_back_warp':[],'bad_head':[],'bad_innner_thigh':[],'bad_shallow':[],'bad_toe':[],'good':[]}

for label in tqdm(labels):
    for single_file in json_files[label]:
        file_path=paths[label]+"/"+single_file
        with open(file_path,'r') as fp:
            data[label].append(json.load(fp))

100%|██████████| 7/7 [00:57<00:00,  8.28s/it]


In [ ]:
data_numpy={}
entire_num=0
for label in labels:
    label_list=[]
    for i in tqdm(range(len(data[label]))):
        my_list=[]
        for j in range(300):
            my_list.append(data[label][i][str(j)]['3d_joint']) 
        label_list.append(my_list)
    label_numpy=np.array(label_list)
    data_numpy[label]=label_numpy
    entire_num+=data_numpy[label].shape[0]
    print(data_numpy[label].shape)
print(entire_num)

100%|██████████| 280/280 [00:00<00:00, 5400.76it/s]


(280, 300, 171)


100%|██████████| 312/312 [00:00<00:00, 2911.53it/s]


(312, 300, 171)


100%|██████████| 272/272 [00:00<00:00, 2459.88it/s]


...

(319, 300, 171)


100%|██████████| 295/295 [00:00<00:00, 7268.94it/s]


(295, 300, 171)


100%|██████████| 293/293 [00:00<00:00, 7715.49it/s]


(293, 300, 171)
2001


## Data Augmentation

In [ ]:
#300 프레임 중 연속된 200프레임,150프레임, 100프레임을 무작위로 추출하여 300프레임으로 늘려줘서 데이터셋에 추가시켜준다.
#이는 다양한 속도의 스쿼트를 학습 시키는데 도움을 준다.

import random
data_200={}
data_150={}
data_100={}
test_samples={}
test_labels=[]

test=0
train=0
addition=0
for label in labels[1:]:
  #test셋은 augmentation data를 포함하지 않은 원본에서 10퍼센트를 뽑아준다. 
  np.random.shuffle(data_numpy[label])
  print("합계",data_numpy[label].shape[0])
  num_test=int(0.1*len(data_numpy[label]))
  test_samples[label]=data_numpy[label][0:num_test,:,:]
  data_numpy[label]=data_numpy[label][num_test:,:,:]
  size=len(data_numpy[label])
  print(test_samples[label].shape[0],data_numpy[label].shape[0])
  test+=test_samples[label].shape[0]
  train+=data_numpy[label].shape[0]
   
  #150 frames augmentation
  a=np.arange(300)
  xp1=np.arange(0,300,2)
  data_150[label]=np.zeros((int(size/3),300,171))
  for i in range(int(size/3)):
    seed_150=random.randint(0,150)
    for j in range(171):
      fp=data_numpy[label][i,seed_150:seed_150+150,j]
      data_150[label][i,:,j]=np.interp(a,xp1,fp)
  addition+=data_150[label].shape[0]


  #200 frames augmentation
  xp3=np.arange(0,600,3)  
  np.random.shuffle(data_numpy[label])
  candidate=np.zeros((int(size/3),600,171))
  data_200[label]=np.zeros((int(size/3),300,171))
  b=np.arange(600)
  for i in range(int(size/3)):
    seed_200=random.randint(0,100)
    for j in range(171):
        fp=data_numpy[label][i,seed_200:seed_200+200,j]
        candidate[i,:,j]=np.interp(b,xp3,fp)
        data_200[label][i,:,j]=candidate[i,::2,j]
  addition+=data_200[label].shape[0]
  
  #100 frames augmentation
  np.random.shuffle(data_numpy[label])
  xp2=np.arange(0,300,3)
  data_100[label]=np.zeros((int(size/3),300,171))
  for i in range(int(size/3)):
    seed_100=random.randint(0,200)
    for j in range(171):
      fp=data_numpy[label][i,seed_100:seed_100+100,j]
      data_100[label][i,:,j]=np.interp(a,xp2,fp)
  addition+=data_100[label].shape[0]
      
print("test",test,"train",train,"addtion",addition,"entrie_num",entire_num)

합계 312
31 281
합계 272
27 245
합계 230
23 207
합계 319
31 288
합계 295
29 266
합계 293
29 264
test 170 train 1551 addtion 1545 entrie_num 2001


##Data Preprocessing

###Split the Dataset

#### 모두 포함

In [ ]:
train_samples={}
valid_samples={}
labels=['bad_back_round','bad_back_warp','bad_head','bad_innner_thigh','bad_shallow','bad_toe','good']
label_encoding={'bad_back_round':[1,0,0,0,0,0,0],'bad_back_warp':[0,1,0,0,0,0,0],'bad_head':[0,0,1,0,0,0,0],'bad_innner_thigh':[0,0,0,1,0,0,0],'bad_shallow':[0,0,0,0,1,0,0],'bad_toe':[0,0,0,0,0,1,0],'good':[0,0,0,0,0,0,1]}
train_labels=[]
valid_labels=[]
train_x=[]
valid_x=[]
test_x=[]
number_data=0




#train: valid= 8:2로 나눠봄
for label in labels:
    np.random.shuffle(data_numpy[label])
    np.random.shuffle(data_150[label])
    np.random.shuffle(data_100[label])
    number_data+=len(data_numpy[label])+len(data_150[label])+len(data_100[label])+len(data_200[label])
    entire_sample=np.concatenate((data_numpy[label],data_150[label],data_100[label],data_200[label]),axis=0)
    num_train=int(0.8*len(entire_sample))
    train_samples[label],valid_samples[label]=entire_sample[:num_train,:,:],entire_sample[num_train:,:,:]



for label in labels:
    #print("----"+label+"-----")
    for i in range(train_samples[label].shape[0]): 
        train_labels.append(label_encoding[label])
        train_x.append(train_samples[label][i,:,:].transpose())
    
    for i in range(valid_samples[label].shape[0]):
        valid_labels.append(label_encoding[label])
        valid_x.append(valid_samples[label][i,:,:].transpose())
    
    for i in range(test_samples[label].shape[0]):
        test_labels.append(label_encoding[label])   
        test_x.append(test_samples[label][i,:,:].transpose()) 

train_x=np.array(train_x)
train_labels=np.array(train_labels)

valid_x=np.array(valid_x)
valid_labels=np.array(valid_labels)
test_x=np.array(test_x)
test_labels=np.array(test_labels)



train_x,train_labels=shuffle(train_x,train_labels)
valid_x,valid_labels=shuffle(valid_x,valid_labels)
test_x,test_labels=shuffle(test_x,test_labels)

train_labels = train_labels.reshape(-1, 1, 1, 7)
valid_labels = valid_labels.reshape(-1, 1, 1, 7)
test_labels=test_labels.reshape((-1,1,1,7))

print("train_x",train_x.shape)
print("train_label",train_labels.shape)

print("test_x",test_x.shape)
print("test_label",test_labels.shape)

print("valid_x",valid_x.shape)
print("valid_label",valid_labels.shape)

print("num_data",number_data)


In [ ]:
a=np.random.randint(1,6)
print(a)

2


#### Bad Back round 제외

In [ ]:
nobbr_train_labels=[]
nobbr_test_labels=[]
nobbr_valid_labels=[]
nobbr_train_x=[]
nobbr_valid_x=[]
nobbr_test_x=[]
for i,label in enumerate(train_labels):
    if(label[0][0][0]==1):
        pass
    elif(label[0][0][1]==1):
        a=np.random.randint(1,6)
        if(a==1):
            nobbr_train_x.append(train_x[i])
            nobbr_train_labels.append(label)
    elif(label[0][0][2]==1):
        a=np.random.randint(1,6)
        if(a==1):
            nobbr_train_x.append(train_x[i])
            nobbr_train_labels.append(label)
    elif(label[0][0][3]==1):
        a=np.random.randint(1,6)
        if(a==1):
            nobbr_train_x.append(train_x[i])
            nobbr_train_labels.append(label)
    elif(label[0][0][4]==1):
        a=np.random.randint(1,6)
        if(a==1):
            nobbr_train_x.append(train_x[i])
            nobbr_train_labels.append(label)
    elif(label[0][0][5]==1):
        a=np.random.randint(1,6)
        if(a==1):
            nobbr_train_x.append(train_x[i])
            nobbr_train_labels.append(label)
    else:
        nobbr_train_x.append(train_x[i])
        nobbr_train_labels.append(label)
for i,label in enumerate(valid_labels):
    if(label[0][0][0]==1):
        pass
    elif(label[0][0][1]==1):
        a=np.random.randint(1,6)
        if(a==1):
            nobbr_valid_x.append(valid_x[i])
            nobbr_valid_labels.append(label)
    elif(label[0][0][2]==1):
        a=np.random.randint(1,6)
        if(a==1):
            nobbr_valid_x.append(valid_x[i])
            nobbr_valid_labels.append(label)
    elif(label[0][0][3]==1):
        a=np.random.randint(1,6)
        if(a==1):
            nobbr_valid_x.append(valid_x[i])
            nobbr_valid_labels.append(label)
    elif(label[0][0][4]==1):
        a=np.random.randint(1,6)
        if(a==1):
            nobbr_valid_x.append(valid_x[i])
            nobbr_valid_labels.append(label)
    elif(label[0][0][5]==1):
        a=np.random.randint(1,6)
        if(a==1):
            nobbr_valid_x.append(valid_x[i])
            nobbr_valid_labels.append(label)
    else:
        nobbr_valid_x.append(valid_x[i])
        nobbr_valid_labels.append(label)
for i,label in enumerate(test_labels):
    if(label[0][0][0]==1):
        pass
    elif(label[0][0][1]==1):
        a=np.random.randint(1,6)
        if(a==1):
            nobbr_test_x.append(test_x[i])
            nobbr_test_labels.append(label)
    elif(label[0][0][2]==1):
        a=np.random.randint(1,6)
        if(a==1):
            nobbr_test_x.append(test_x[i])
            nobbr_test_labels.append(label)
    elif(label[0][0][3]==1):
        a=np.random.randint(1,6)
        if(a==1):
            nobbr_test_x.append(test_x[i])
            nobbr_test_labels.append(label)
    elif(label[0][0][4]==1):
        a=np.random.randint(1,6)
        if(a==1):
            nobbr_test_x.append(test_x[i])
            nobbr_test_labels.append(label)
    elif(label[0][0][5]==1):
        a=np.random.randint(1,6)
        if(a==1):
            nobbr_test_x.append(test_x[i])
            nobbr_test_labels.append(label)
    else:
        nobbr_test_x.append(test_x[i])
        nobbr_test_labels.append(label)

In [ ]:
nobbr_train_x=np.array(nobbr_train_x)
nobbr_valid_x=np.array(nobbr_valid_x)
nobbr_test_x=np.array(nobbr_test_x)
nobbr_train_labels=np.array(nobbr_train_labels)
nobbr_test_labels=np.array(nobbr_test_labels)
nobbr_valid_labels=np.array(nobbr_valid_labels)

In [ ]:
print(nobbr_train_x.shape,nobbr_train_labels.shape)
print(nobbr_test_x.shape,nobbr_test_labels.shape)
print(nobbr_valid_x.shape,nobbr_valid_labels.shape)

(813, 171, 300) (813, 1, 1, 7)
(51, 171, 300) (51, 1, 1, 7)
(208, 171, 300) (208, 1, 1, 7)


#### Good label

In [ ]:
train_good_labels=[]
valid_good_labels=[]
np.random.shuffle(data_numpy['good'])
np.random.shuffle(data_150['good'])
np.random.shuffle(data_100['good'])
number_good=len(data_numpy['good'])+len(data_150['good'])+len(data_100['good'])+len(data_200['good'])
good_samples=np.concatenate((data_numpy['good'],data_150['good'],data_100['good'],data_200['good']),axis=0)
num_train=int(0.8*len(good_samples))
train_good,valid_good=good_samples[:num_train,:,:],good_samples[num_train:,:,:]



for i in range(train_good.shape[0]): 
    train_good_labels.append(label_encoding['good'])
for i in range(valid_good.shape[0]):
    valid_good_labels.append(label_encoding['good'])

train_good=np.array(train_good)
train_good_labels=np.array(train_good_labels)
valid_good=np.array(valid_good)
valid_good_labels=np.array(valid_good_labels)


train_good,train_good_labels=shuffle(train_good,train_good_labels)
valid_good,valid_good_labels=shuffle(valid_good,valid_good_labels)
train_good_labels = train_good_labels.reshape(-1, 1, 1, 7)
valid_good_labels=valid_good_labels.reshape(-1,1,1,7)


print(train_good.shape,valid_good.shape,train_good_labels.shape,valid_good_labels.shape)



(422, 300, 171) (106, 300, 171) (422, 1, 1, 7) (106, 1, 1, 7)


In [ ]:
train_good=train_good.reshape((-1,171,300))
valid_good=valid_good.reshape((-1,171,300))
print(train_good.shape)
print(valid_good.shape)

(422, 171, 300)
(106, 171, 300)


###Normalization

In [ ]:
from sklearn.preprocessing import MinMaxScaler



for i in range(len(nobbr_train_x)):
  scaler = MinMaxScaler()
  matrix=nobbr_train_x[i]
  matrix_one_column=matrix.reshape((-1,1))
  scaler.fit(matrix_one_column)
  normalized_arr = scaler.transform(matrix_one_column)
  nobbr_train_x[i]=normalized_arr.reshape((matrix.shape))

for i in range(len(nobbr_valid_x)):
  scaler = MinMaxScaler()
  matrix=nobbr_valid_x[i]
  matrix_one_column=matrix.reshape((-1,1))
  scaler.fit(matrix_one_column)
  normalized_arr = scaler.transform(matrix_one_column)
  nobbr_valid_x[i]=normalized_arr.reshape((matrix.shape))

for i in range(len(nobbr_test_x)):
  scaler = MinMaxScaler()
  matrix=nobbr_test_x[i]
  matrix_one_column=matrix.reshape((-1,1))
  scaler.fit(matrix_one_column)
  normalized_arr = scaler.transform(matrix_one_column)
  nobbr_test_x[i]=normalized_arr.reshape((matrix.shape))


In [ ]:
for i in range(len(train_good)):
  scaler = MinMaxScaler()
  matrix=train_good[i]
  matrix_one_column=matrix.reshape((-1,1))
  scaler.fit(matrix_one_column)
  normalized_arr = scaler.transform(matrix_one_column)
  train_good[i]=normalized_arr.reshape((matrix.shape))

for i in range(len(valid_good)):
  scaler = MinMaxScaler()
  matrix=valid_good[i]
  matrix_one_column=matrix.reshape((-1,1))
  scaler.fit(matrix_one_column)
  normalized_arr = scaler.transform(matrix_one_column)
  valid_good[i]=normalized_arr.reshape((matrix.shape))


### Delete unneccessary files

In [ ]:
del train_samples
del valid_samples
del test_samples
del data_100
del data_150
del data_200 
del data_numpy

In [ ]:
del data

In [ ]:
del paths
del json_files

### test,train,valid 저장

In [ ]:
np.save(ftrain_x,train_x)
np.save(ftest_x,test_x)
np.save(fvalid_x,valid_x)
np.save(ftrain_labels,train_labels)
np.save(ftest_labels,test_labels)
np.save(fvalid_labels,valid_labels)

In [ ]:
np.save(ftrain_x,nobbr_train_x)
np.save(ftest_x,nobbr_test_x)
np.save(fvalid_x,nobbr_valid_x)
np.save(ftrain_labels,nobbr_train_labels)
np.save(ftest_labels,nobbr_test_labels)
np.save(fvalid_labels,nobbr_valid_labels)

In [ ]:
np.save("goodtrain",train_good)
np.save("goodvalid",valid_good)
np.save("goodtrainlabel",train_good_labels)
np.save("goodvalidlabel",valid_good_labels)

# Load the Train, Valid, Test Dataset 

In [4]:
ftrain_x="train_x" 
fvalid_x="valid_x"
ftest_x="test_x"
ftrain_labels ="train_labels"
fvalid_labels ="valid_labels"
ftest_labels="test_labels"

In [5]:
train_x=np.load(ftrain_x+".npy")
valid_x=np.load(fvalid_x+".npy")
test_x=np.load(ftest_x+".npy")
train_labels=np.load(ftrain_labels+".npy")
valid_labels=np.load(fvalid_labels+".npy")
test_labels=np.load(ftest_labels+".npy")

In [ ]:
nobbr_train_x=np.load(ftrain_x+".npy")
nobbr_valid_x=np.load(fvalid_x+".npy")
nobbr_test_x=np.load(ftest_x+".npy")
nobbr_train_labels=np.load(ftrain_labels+".npy")
nobbr_valid_labels=np.load(fvalid_labels+".npy")
nobbr_test_labels=np.load(ftest_labels+".npy")

In [6]:
print(test_labels[68])

[[[0 0 1 0 0 0 0]]]


## Model

### Define Resnet block 

In [ ]:
def resblock(frames):
    #resblock에 들어온 input_layer
    input_layer=Input(shape=frames)

    
    res_layer=Conv2D(filters=256,kernel_size=(1,3),strides=(1,2),padding='same')(input_layer)
    res_layer=BatchNormalization()(res_layer)
    
    
    conv1d_layer1=Conv2D(filters=256,kernel_size=(1,3),strides=(1,2),padding='same',kernel_regularizer=regularizers.L2(0.01))(input_layer)
    conv1d_layer1=BatchNormalization()(conv1d_layer1)
    conv1d_layer1=tf.keras.layers.ReLU()(conv1d_layer1)


    conv1d_layer1=Conv2D(filters=256,kernel_size=(1,3),strides=(1,1),padding='same')(conv1d_layer1)
    conv1d_layer1=BatchNormalization()(conv1d_layer1)
    conv1d_layer1=tf.keras.layers.ReLU()(conv1d_layer1)


    conv1d_layer1=Conv2D(filters=256,kernel_size=(1,3),strides=(1,1),padding='same')(conv1d_layer1)
    conv1d_layer1=BatchNormalization()(conv1d_layer1)
    conv1d_layer1=tf.keras.layers.ReLU()(conv1d_layer1)

    conv1d_layer1=Dropout(0.2)(conv1d_layer1)
    
   
    output=tf.keras.layers.Add()([res_layer,conv1d_layer1])
    block=tf.keras.Model(inputs=input_layer,outputs=output)
    
    return block

### Reference Model

In [ ]:
def create_model():
    inputs=tf.keras.Input(shape=(171,300,1))
    layer1 = Conv2D(filters=256, kernel_size=(1, 3), strides=(1, 2),padding='same',kernel_regularizer=regularizers.L2(0.01))(inputs)
    layer1=BatchNormalization()(layer1)
    layer1=tf.keras.layers.ReLU()(layer1)

    # (B,171,300,1) matrix를 (B.300,1) matrix 171 개로 찢어준다.

    layer2=resblock((171,150,256))(layer1)
    layer3=resblock((171,75,256))(layer2)
    layer4=resblock((171,38,256))(layer3)
    layer5=resblock((171,19,256))(layer4)

    layer6= AveragePooling2D(pool_size=(171,10),strides=(1,1),padding='valid')(layer5)

    layer6=Dropout(0.2)(layer6)
    layer7=Conv2D(filters=7,kernel_size=(1,1),strides=(1,1))(layer6)

    outputs=Softmax()(layer7)

    my_model=tf.keras.Model(inputs=inputs,outputs=outputs)

    return my_model

### Dense Model

In [ ]:
def create_dense():
    inputs=tf.keras.Input(shape=(171,300,1))
    layer1 = Conv2D(filters=256, kernel_size=(1, 3), strides=(1, 2),padding='same',kernel_regularizer=regularizers.L2(0.01))(inputs)
    layer1=BatchNormalization()(layer1)
    layer1=tf.keras.layers.ReLU()(layer1)

    # (B,171,300,1) matrix를 (B.300,1) matrix 171 개로 찢어준다.

    layer2=resblock((171,150,256))(layer1)
    layer3=resblock((171,75,256))(layer2)
    layer4=resblock((171,38,256))(layer3)
    layer5=resblock((171,19,256))(layer4)

    layer6= tf.keras.layers.Flatten()(layer5)
    print(layer6.shape)

    layer6=Dropout(0.2)(layer6)
    layer7=tf.keras.layers.Dense(7)(layer6)
    layer7=Softmax()(layer7)

    outputs=tf.keras.layers.Reshape((1,1,7), input_shape=(7,))(layer7)
  
    print(outputs.shape)
    my_model=tf.keras.Model(inputs=inputs,outputs=outputs)

    return my_model

In [ ]:
d_model.summary()

Model: "model_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 171, 300, 1)]     0         
                                                                 
 conv2d_17 (Conv2D)          (None, 171, 150, 256)     1024      
                                                                 
 batch_normalization_17 (Bat  (None, 171, 150, 256)    1024      
 chNormalization)                                                
                                                                 
 re_lu_13 (ReLU)             (None, 171, 150, 256)     0         
                                                                 
 model_5 (Functional)        (None, 171, 75, 256)      791552    
                                                                 
 model_6 (Functional)        (None, 171, 38, 256)      791552    
                                                           

## Train the Model


### Training Reference Model

In [ ]:


initial_learning_rate=0.01
lr_schedule=tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=100000,
    decay_rate=0.96,
    staircase=True
)


# 체크포인트 콜백 만들기

callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                           patience=5),
             tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                             monitor='val_loss',
                                             save_weights_only=True,
                                             save_best_only=True,verbose=2)]

my_model=create_model()
my_model.compile(optimizer=tf.keras.optimizers.Adadelta(learning_rate=lr_schedule),loss='categorical_crossentropy',metrics=['accuracy'])
my_model.summary()

In [ ]:
my_model.fit(x=train_x,y=train_labels,batch_size=16,epochs=60,validation_data=(valid_x,valid_labels),verbose=2, callbacks = [callbacks])

Epoch 1/60


#### CheckPoint Training

In [ ]:
# checkpoint_path로 부터 model을 받아와서 loading
test_labels=test_labels.reshape((-1,1,1,7))
initial_learning_rate=0.006
lr_schedule=tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=100000,
    decay_rate=0.96,
    staircase=True
)


my_model=create_model()
my_model.compile(optimizer=tf.keras.optimizers.Adadelta(learning_rate=lr_schedule),loss='categorical_crossentropy',metrics=['accuracy'])
my_model.load_weights(checkpoint_path)
loss,acc = my_model.evaluate(test_x,test_labels, verbose=2)

7/7 - 3s - loss: 4.7772 - accuracy: 0.7222 - 3s/epoch - 489ms/step


In [ ]:
my_model.summary()

In [ ]:
#checkpoint2에다가 저장함. checkpoint2에는 69퍼센트까지 늘린 모델 저장되어 있음.
callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                           patience=8),
             tf.keras.callbacks.ModelCheckpoint(checkpoint_path2,
                                             monitor='val_loss',
                                             save_weights_only=True,
                                             save_best_only=True,verbose=2),
             ]

my_model.fit(x=train_x,y=train_labels,batch_size=16,epochs=60,validation_data=(valid_x,valid_labels),verbose=2, callbacks = [callbacks])


### Training Dense Model

In [ ]:
d_model=create_dense()
d_model.summary()

In [ ]:
initial_learning_rate=0.01
lr_schedule=tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=1000000,
    decay_rate=0.96,
    staircase=True
)


# 체크포인트 콜백 만들기

callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                           patience=5),
             tf.keras.callbacks.ModelCheckpoint(checkpoint_path3,
                                             monitor='val_loss',
                                             save_weights_only=True,
                                             save_best_only=True,verbose=2)]

d_model.compile(optimizer=tf.keras.optimizers.Adadelta(learning_rate=lr_schedule),loss='categorical_crossentropy',metrics=['accuracy'])
d_model.fit(x=train_x,y=train_labels,batch_size=16,epochs=60,validation_data=(valid_x,valid_labels),verbose=2, callbacks = [callbacks])

Epoch 1/60

Epoch 1: val_loss improved from inf to 11.66981, saving model to ck3/cp.ckpt
180/180 - 244s - loss: 11.4831 - accuracy: 0.6004 - val_loss: 11.6698 - val_accuracy: 0.4931 - 244s/epoch - 1s/step
Epoch 2/60

Epoch 2: val_loss improved from 11.66981 to 11.64566, saving model to ck3/cp.ckpt
180/180 - 195s - loss: 11.0830 - accuracy: 0.6803 - val_loss: 11.6457 - val_accuracy: 0.5305 - 195s/epoch - 1s/step
Epoch 3/60

Epoch 3: val_loss improved from 11.64566 to 11.63276, saving model to ck3/cp.ckpt
180/180 - 194s - loss: 10.9406 - accuracy: 0.7179 - val_loss: 11.6328 - val_accuracy: 0.5166 - 194s/epoch - 1s/step
Epoch 4/60

Epoch 4: val_loss improved from 11.63276 to 11.44734, saving model to ck3/cp.ckpt
180/180 - 195s - loss: 10.7726 - accuracy: 0.7467 - val_loss: 11.4473 - val_accuracy: 0.5471 - 195s/epoch - 1s/step
Epoch 5/60

Epoch 5: val_loss improved from 11.44734 to 11.38552, saving model to ck3/cp.ckpt
180/180 - 195s - loss: 10.6360 - accuracy: 0.7804 - val_loss: 11.3855 -

In [ ]:
d_model.save("cid_epoch40_72_real.h5")

#### CheckPoint Training

In [ ]:
# checkpoint_path4로 부터 model을 받아와서 loading
nobbr_test_labels=nobbr_test_labels.reshape((-1,1,1,7))
initial_learning_rate=0.01
lr_schedule=tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=100000,
    decay_rate=0.96,
    staircase=True
)


d_model=create_dense()
d_model.compile(optimizer=tf.keras.optimizers.Adadelta(learning_rate=lr_schedule),loss='categorical_crossentropy',metrics=['accuracy'])
d_model.load_weights(checkpoint_path3)
d_model.evaluate(test_x,test_labels,verbose=2)

(None, 437760)
(None, 1, 1, 7)
7/7 - 188s - loss: 3.1258 - accuracy: 0.7323 - 188s/epoch - 27s/step


[3.1258485317230225, 0.7323232293128967]

In [ ]:
d_model.save("0304_goodemph60.h5")

NameError: ignored

In [ ]:
!pip install tensorflowjs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 KB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.1/210.1 KB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 KB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.9/154.9 KB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 49.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.2/74.2 KB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.0/239.0 KB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 65.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 KB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.9/87.9 KB 11.2 MB/s eta 0:00:00
  Attempting unin

In [ ]:
!tensorflowjs_converter --input_format keras --weight_shard_size_bytes 30000000 "./0304_goodemph60.h5" "./"

2023-03-04 05:16:56.999567: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-04 05:16:56.999788: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-04 05:16:56.999819: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.


In [ ]:
#checkpoint3에다가 nobbr을 더 잘 detect 하는 저장함. checkpoint4에는 전체로 training 한 모델 저장되어 있음.
callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                           patience=5),
             tf.keras.callbacks.ModelCheckpoint(checkpoint_path3,
                                             monitor='val_loss',
                                             save_weights_only=True,
                                             save_best_only=True,verbose=2),
             ]

d_model.fit(x=nobbr_train_x,y=nobbr_train_labels,batch_size=16,epochs=60,validation_data=(nobbr_valid_x,nobbr_valid_labels),verbose=2, callbacks = [callbacks])

Epoch 1/60

Epoch 1: val_loss improved from inf to 6.12028, saving model to ck3/cp.ckpt
51/51 - 70s - loss: 4.4225 - accuracy: 1.0000 - val_loss: 6.1203 - val_accuracy: 0.6154 - 70s/epoch - 1s/step
Epoch 2/60

Epoch 2: val_loss improved from 6.12028 to 6.08312, saving model to ck3/cp.ckpt
51/51 - 61s - loss: 4.3854 - accuracy: 0.9988 - val_loss: 6.0831 - val_accuracy: 0.6058 - 61s/epoch - 1s/step
Epoch 3/60

Epoch 3: val_loss did not improve from 6.08312
51/51 - 60s - loss: 4.3460 - accuracy: 1.0000 - val_loss: 6.1720 - val_accuracy: 0.5769 - 60s/epoch - 1s/step
Epoch 4/60

Epoch 4: val_loss improved from 6.08312 to 6.01823, saving model to ck3/cp.ckpt
51/51 - 60s - loss: 4.2964 - accuracy: 1.0000 - val_loss: 6.0182 - val_accuracy: 0.6010 - 60s/epoch - 1s/step
Epoch 5/60

Epoch 5: val_loss improved from 6.01823 to 5.92493, saving model to ck3/cp.ckpt
51/51 - 60s - loss: 4.2450 - accuracy: 1.0000 - val_loss: 5.9249 - val_accuracy: 0.6106 - 60s/epoch - 1s/step
Epoch 6/60

Epoch 6: val_lo

KeyboardInterrupt: ignored

## Evaluation

In [ ]:
d_model.evaluate(test_x,test_labels,verbose=2)

7/7 - 3s - loss: 3.0348 - accuracy: 0.7222 - 3s/epoch - 403ms/step


[3.0348150730133057, 0.7222222089767456]

In [ ]:
d_model.evaluate(valid_x,valid_labels,verbose=2)

23/23 - 11s - loss: 6.6782 - accuracy: 0.5803 - 11s/epoch - 462ms/step


[6.678152084350586, 0.5803323984146118]

##Grad Cam

In [ ]:
d_model.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 171, 300, 1)]     0         
                                                                 
 conv2d (Conv2D)             (None, 171, 150, 256)     1024      
                                                                 
 batch_normalization (BatchN  (None, 171, 150, 256)    1024      
 ormalization)                                                   
                                                                 
 re_lu (ReLU)                (None, 171, 150, 256)     0         
                                                                 
 model (Functional)          (None, 171, 75, 256)      791552    
                                                                 
 model_1 (Functional)        (None, 171, 38, 256)      791552    
                                                           

In [ ]:
#사전 학습된 신경망 모델을 불러오고 구조 확인
    #지정된 영상을 불러와 크기 조정하고 화면에 디스플레이
test_x = np.load("test_x.npy")

index = np.random.randint(len(test_x))
print(index)
test_gradcam = test_x[int(index)]
print(test_gradcam.shape)

plt.imshow(test_gradcam)
plt.show()
test_gradcam = test_gradcam.reshape((171, 300, 1))
#영상을 신경망 형태로 변환
x=test_gradcam
x=np.expand_dims(x,axis=0)

#인식을 시도하고 top-5결과를 출력
preds = d_model.predict(x)
#신경망 모델의 특 징  추출 부분에서 마지막 층을 지정
#특징 추출 부분만으로 구성된 model_1만들기

input1 = keras.Input(shape=(171,300,1))
last_conv_layer = model.get_layer("conv2d")(input1)
last_conv_layer = model.get_layer("batch_normalization")(last_conv_layer)
last_conv_layer = model.get_layer("re_lu")(last_conv_layer)
last_conv_layer = model.get_layer("model")(last_conv_layer)
last_conv_layer = model.get_layer("model_1")(last_conv_layer)
last_conv_layer = model.get_layer("model_2")(last_conv_layer)
last_conv_layer = model.get_layer("model_3")(last_conv_layer)

model_1 = keras.Model(input1, last_conv_layer)
print(model_1.summary())
print(x.shape)
#분류 (전역평균풀링 또는 완전연결층) 부분만으로 구성된 model__2만들기
input_2 = keras.Input(shape=model_1.output.shape[1:])
print(input_2.shape)
x_2 = model.get_layer("flatten")(input_2)
x_2 = model.get_layer("dense")(x_2)
x_2 = model.get_layer("softmax")(x_2)
x_2 = model.get_layer("reshape")(x_2)
model_2 =keras.Model(input_2,x_2)
print(model_2.summary())
#GradientTape함수를 이용한 그레디언트 계산
with tf.GradientTape() as tape:
output_1 = model_1(x)
print(output_1.shape)
tape.watch(output_1) #마지막 층으로 미분하기 위한 준비
preds = model_2(output_1)
print(preds.shape)
class_id = tf.argmax(preds[0,0,0,:])
print(class_id)
output_2 = preds[:,:,:,class_id]

grads = tape.gradient(output_2, output_1) #그레디언트 계산
pooled_grads = tf.reduce_mean(grads,axis=(0,1,2)) #식5 적용

output_1 = output_1.numpy()[0]
pooled_grads = pooled_grads.numpy()
for i in range(pooled_grads.shape[-1]):
    output_1[:,:,i]*=pooled_grads[i]
heatmap=np.mean(output_1, axis=-1)

heatmap =np.maximum(heatmap, 0)/np.max(heatmap) #정규화
plt.matshow(heatmap)

#열지도를 입력 영상에 씌움
img=test_gradcam
heatmap=np.uint8(255*heatmap) # [0,255]로 변환

jet = cm.get_cmap("jet") #jet 컬러맵으로 표시
color = jet(np.arange(256))[:,:3]
color_heatmap = color[heatmap]

color_heatmap = keras.preprocessing.image.array_to_img(color_heatmap)
color_heatmap = color_heatmap.resize((img.shape[1], img.shape[0]))
color_heatmap = keras.preprocessing.image.img_to_array(color_heatmap)

overlay_img= color_heatmap*0.4+img #덧씌움
overlay_img = keras.preprocessing.image.array_to_img(overlay_img)
plt.matshow(overlay_img)
plt.show()

In [ ]:
index = np.random.randint(len(test_x))
print(index)
test_gradcam = test_x[int(index)].reshape((-1,171,300,1))